# Fashion MNISTデータセットの学習(CNN手法)

モジュールのインポート

In [1]:
from time import time   # 時間計測を行うモジュール

import numpy as np      # Pythonで配列を扱うためのモジュール
import torch            # 深層学習のモジュール（PyTorch）
import torch.nn as nn

import torchvision      # PyTorchで画像認識に使用するネットワークやデータセットを利用するためのモジュール
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision.transforms import ToTensor

import torchsummary     # PyTorchで作成したネットワークモデルの詳細を表示するモジュール

グラボが使用可能かどうかの確認

In [2]:
use_cuda = torch.cuda.is_available()
print(use_cuda)

True


データセットの読み込みとニューラルネットワーク用への調整

In [3]:
Batch_size = 100

training_data = datasets.FashionMNIST(
    root="../data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="../data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = torch.utils.data.DataLoader(training_data, batch_size=Batch_size, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=Batch_size, shuffle=True)

print(type(training_data.data), type(training_data.targets))
print(type(test_data.data), type(test_data.targets))

print(training_data.data.size(), training_data.targets.size())
print(test_data.data.size(), test_data.targets.size())

<class 'torch.Tensor'> <class 'torch.Tensor'>
<class 'torch.Tensor'> <class 'torch.Tensor'>
torch.Size([60000, 28, 28]) torch.Size([60000])
torch.Size([10000, 28, 28]) torch.Size([10000])


## ネットワークモデルの定義
今回は畳み込み層が2層のニューラルネットワークを作る

In [8]:
class CNN(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 48, kernel_size=3, stride=1, padding=1)
        # self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.l1 = nn.Linear(7*7*48, 1024)
        self.l2 = nn.Linear(1024, 1024)
        self.l3 = nn.Linear(1024, 10)
        self.act = nn.LeakyReLU()
        self.pool = nn.MaxPool2d(2,2)
    
    def forward(self, x):
        x = self.act(self.conv1(x))
        x = self.pool(x)
        x = self.act(self.conv2(x))
        # x = self.act(self.conv3(x))
        x = self.pool(x)
        x = x.view(x.size()[0], -1)
        x = self.act(self.l1(x))
        x = self.act(self.l2(x))
        x = self.l3(x)
        return x

In [9]:
### ネットワークの作成
model = CNN()
if use_cuda:      # GPUが使用できる場合は
    model.cuda()  # ネットワークの演算をGPUで行うように設定変更 (デフォルトはCPU)

### 最適化手法の設定 lr: 学習率, momentum: モーメンタム (慣性項のパラメータ)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

### 定義したモデルの情報を表示
if use_cuda:
    torchsummary.summary(model, (1, 28, 28), device='cuda')  # GPUで演算する設定の場合
else:
    torchsummary.summary(model, (1, 28, 28), device='cpu')   # CPUで演算する設定の場合

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
         LeakyReLU-2           [-1, 16, 28, 28]               0
         MaxPool2d-3           [-1, 16, 14, 14]               0
            Conv2d-4           [-1, 48, 14, 14]           6,960
         LeakyReLU-5           [-1, 48, 14, 14]               0
         MaxPool2d-6             [-1, 48, 7, 7]               0
            Linear-7                 [-1, 1024]       2,409,472
         LeakyReLU-8                 [-1, 1024]               0
            Linear-9                 [-1, 1024]       1,049,600
        LeakyReLU-10                 [-1, 1024]               0
           Linear-11                   [-1, 10]          10,250
Total params: 3,476,442
Trainable params: 3,476,442
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forw

In [11]:
epoch_num=15# 誤差関数の設定 (必要に応じて誤差関数の計算もGPUで行うように設定変更)
criterion = nn.CrossEntropyLoss()
if use_cuda:
    criterion.cuda()

# ネットワークを学習モードへ変更
model.train()

# 学習の実行
train_start = time()
for epoch in range(1, epoch_num+1):   # epochのforループ
    # 1 epochの学習中の誤差・学習画像が正解した数をカウントする変数を初期化
    sum_loss = 0.0
    count = 0

    for image, label in train_dataloader:  # 1 epoch内のforループ (iterationのループ)

        if use_cuda:  # GPUで計算する場合は，データもGPUメモリ上へ移動させる
            image = image.cuda()
            label = label.cuda()

        y = model(image)  # データの入力と結果の出力

        # 誤差計算とbackpropagation, パラメータの更新
        loss = criterion(y, label)
        model.zero_grad()
        loss.backward()
        optimizer.step()

        # 学習経過を確認するための処理
        sum_loss += loss.item()
        pred = torch.argmax(y, dim=1)
        count += torch.sum(pred == label)

    # 1 epoch終了時点での誤差の平均値，学習データに対する認識精度, 学習開始からの経過時間を表示
    print("epoch: {}, mean loss: {}, mean accuracy: {}, elapsed time: {}".format(epoch, sum_loss/len(train_dataloader), count.item()/len(training_data), time() - train_start))

epoch: 1, mean loss: 0.20142304466416439, mean accuracy: 0.9246833333333333, elapsed time: 12.901903629302979
epoch: 2, mean loss: 0.18794585715979337, mean accuracy: 0.9292666666666667, elapsed time: 25.533681631088257
epoch: 3, mean loss: 0.17739280263582866, mean accuracy: 0.9332, elapsed time: 38.20395517349243
epoch: 4, mean loss: 0.16928910439834, mean accuracy: 0.9364, elapsed time: 50.911508560180664
epoch: 5, mean loss: 0.1605418050289154, mean accuracy: 0.9395, elapsed time: 63.76161289215088
epoch: 6, mean loss: 0.1494046123760442, mean accuracy: 0.9437833333333333, elapsed time: 76.69737434387207
epoch: 7, mean loss: 0.14163035905299087, mean accuracy: 0.9467166666666667, elapsed time: 89.61261820793152
epoch: 8, mean loss: 0.1286285556294024, mean accuracy: 0.9516333333333333, elapsed time: 102.53581762313843
epoch: 9, mean loss: 0.12058911205890278, mean accuracy: 0.95445, elapsed time: 115.6286849975586
epoch: 10, mean loss: 0.11054365411090354, mean accuracy: 0.95853333